In [16]:
import requests
import datetime
import pandas as pd

In [17]:
import pandas as pd
import numpy as np
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.figure_factory as ff
import plotly.offline as py
plotly.tools.set_config_file(world_readable=False,
                             sharing='private')
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import Scatter, Figure, Layout
init_notebook_mode(connected=True)

In [18]:
# Pretty print the JSON
import uuid
from IPython.display import display_javascript, display_html, display
import json

class RenderJSON(object):
    def __init__(self, json_data):
        if isinstance(json_data, dict):
            self.json_str = json.dumps(json_data)
        else:
            self.json_str = json_data
        self.uuid = str(uuid.uuid4())

    def _ipython_display_(self):
        display_html('<div id="{}" style="height: 600px; width:100%;"></div>'.format(self.uuid), raw=True)
        display_javascript("""
        require(["https://rawgit.com/caldwell/renderjson/master/renderjson.js"], function() {
        document.getElementById('%s').appendChild(renderjson(%s))
        });
        """ % (self.uuid, self.json_str), raw=True)

# CryptoCompare Mini API

## Current Price

In [19]:
def price(symbol, comparison_symbols=['USD'], exchange=''):
    url = 'https://min-api.cryptocompare.com/data/price?fsym={}&tsyms={}'\
            .format(symbol.upper(), ','.join(comparison_symbols).upper())
    if exchange:
        url += '&e={}'.format(exchange)
    page = requests.get(url)
    data = page.json()
    return data

In [20]:
price('LTC', exchange='Coinbase')

{'USD': 57.5}

In [21]:
price('NEO', ['BTC', 'ETH', 'USD'])

{'BTC': 0.002758, 'ETH': 0.08076, 'USD': 18.29}

In [22]:
price('ZRX')

{'USD': 0.765}

In [23]:
def price2(symbol, comparison_symbols=['USD'], exchange=''):
    url = 'https://min-api.cryptocompare.com/data/pricemulti?fsyms=HPB,AST&tsyms=USD'\
            .format(symbol.upper(), ','.join(comparison_symbols).upper())
    if exchange:
        url += '&e={}'.format(exchange)
    page = requests.get(url)
    d = page.json()
    return d


In [24]:
pair=price2('pairs')
pair

{'HPB': {'USD': 0.7812}, 'AST': {'USD': 0.09315}}

In [25]:
print(pair)

{'HPB': {'USD': 0.7812}, 'AST': {'USD': 0.09315}}


In [26]:
type(pair)

dict

In [27]:
new=pd.DataFrame([pair])
new

,AST,HPB
0,{'USD': 0.09315},{'USD': 0.7812}


In [28]:
another=pd.DataFrame.from_dict(pair,orient='index')
another

,USD
AST,0.09315
HPB,0.78120


In [29]:
token_plot = go.Bar(x=another.index, y=another['USD'])
iplot([token_plot])

## Daily Historical Price

In [30]:
def daily_price_historical(symbol, comparison_symbol, limit=1, aggregate=1, exchange='', allData='true'):
    url = 'https://min-api.cryptocompare.com/data/histoday?fsym={}&tsym={}&limit={}&aggregate={}&allData={}'\
            .format(symbol.upper(), comparison_symbol.upper(), limit, aggregate, allData)
    if exchange:
        url += '&e={}'.format(exchange)
    page = requests.get(url)
    data = page.json()['Data']
    df = pd.DataFrame(data)
    df['timestamp'] = [datetime.datetime.fromtimestamp(d) for d in df.time]
    return df

In [31]:
neo= daily_price_historical('NEO', 'USD')
print('Max length = %s' % len(neo))
print('Max time = %s' % (neo.timestamp.max() - neo.timestamp.min()))
neo

Max length = 432
Max time = 431 days 00:00:00


,close,high,low,open,time,volumefrom,volumeto,timestamp
0,14.12,15.80,10.21,10.50,1501891200,167756.22,2170085.32,2017-08-04 20:00:00
1,15.97,17.25,13.30,14.12,1501977600,150858.48,2340063.31,2017-08-05 20:00:00
2,19.49,19.93,17.41,18.20,1502064000,97984.60,1840059.23,2017-08-06 20:00:00
3,18.25,20.29,15.68,19.49,1502150400,290834.95,5170450.88,2017-08-07 20:00:00
4,22.60,23.00,17.26,18.25,1502236800,190636.74,3816168.88,2017-08-08 20:00:00
5,37.60,41.72,21.51,22.60,1502323200,413255.22,11987790.71,2017-08-09 20:00:00
6,34.10,40.99,27.75,37.60,1502409600,401554.82,13792663.98,2017-08-10 20:00:00
7,37.72,38.56,30.70,34.10,1502496000,147955.00,5043840.57,2017-08-11 20:00:00
8,48.50,56.69,35.31,37.72,1502582400,319807.81,14965636.31,2017-08-12 20:00:00
9,48.32,54.80,45.40,48.50,1502668800,230083.03,11403353.70,2017-08-13 20:00:00


In [32]:
neo.to_csv('neo.csv')

In [33]:
dbc= daily_price_historical('DBC', 'USD')
print('Max length = %s' % len(dbc))
print('Max time = %s' % (dbc.timestamp.max() - dbc.timestamp.min()))
dbc

Max length = 279
Max time = 278 days 01:00:00


,close,high,low,open,time,volumefrom,volumeto,timestamp
0,0.233600,0.247400,0.183100,0.210200,1515110400,6665023.59,1556949.51,2018-01-04 19:00:00
1,0.295900,0.295900,0.217400,0.238500,1515196800,22822005.83,6753031.53,2018-01-05 19:00:00
2,0.382300,0.388500,0.255300,0.279400,1515283200,23350138.81,8926758.07,2018-01-06 19:00:00
3,0.447800,0.447800,0.344000,0.352700,1515369600,18821374.13,8428211.34,2018-01-07 19:00:00
4,0.665300,0.680000,0.397900,0.432600,1515456000,20075735.14,13356386.59,2018-01-08 19:00:00
5,0.479700,0.684800,0.408500,0.682600,1515542400,22648055.46,10864272.20,2018-01-09 19:00:00
6,0.333100,0.439200,0.302100,0.434000,1515628800,22356455.39,7446935.29,2018-01-10 19:00:00
7,0.353600,0.440000,0.311400,0.338800,1515715200,16563446.38,5856834.64,2018-01-11 19:00:00
8,0.332300,0.382900,0.226000,0.363300,1515801600,19958002.78,6632044.32,2018-01-12 19:00:00
9,0.265100,0.335500,0.222700,0.313800,1515888000,13395250.15,3551080.81,2018-01-13 19:00:00


In [34]:
dbc.to_csv('dbc.csv')

In [35]:
acat= daily_price_historical('ACAT', 'USD')
print('Max length = %s' % len(acat))
print('Max time = %s' % (acat.timestamp.max() - acat.timestamp.min()))
acat.tail()


Max length = 197
Max time = 196 days 00:00:00


,close,high,low,open,time,volumefrom,volumeto,timestamp
192,0.000831,0.000988,0.000741,0.000836,1538784000,6.494253e+07,53973.74,2018-10-05 20:00:00
193,0.000880,0.001015,0.000812,0.000835,1538870400,1.339340e+08,117861.92,2018-10-06 20:00:00
194,0.000853,0.000917,0.000826,0.000894,1538956800,4.983610e+06,4251.52,2018-10-07 20:00:00
195,0.000821,0.000908,0.000773,0.000846,1539043200,2.065350e+06,1696.07,2018-10-08 20:00:00
196,0.000894,0.000918,0.000788,0.000812,1539129600,4.927866e+06,4325.19,2018-10-09 20:00:00


In [36]:

acat

,close,high,low,open,time,volumefrom,volumeto,timestamp
0,0.006020,0.006332,0.005619,0.006198,1522195200,3.960184e+06,23840.31,2018-03-27 20:00:00
1,0.005144,0.005336,0.004837,0.005183,1522281600,3.500197e+06,18005.02,2018-03-28 20:00:00
2,0.005080,0.005277,0.004805,0.005277,1522368000,2.378160e+06,12081.05,2018-03-29 20:00:00
3,0.005202,0.005241,0.004926,0.005084,1522454400,8.380990e+05,4359.79,2018-03-30 20:00:00
4,0.005077,0.005114,0.004281,0.005001,1522540800,1.705390e+06,8658.27,2018-03-31 20:00:00
5,0.005039,0.005232,0.004847,0.005155,1522627200,6.030551e+05,3038.79,2018-04-01 20:00:00
6,0.005657,0.005698,0.005324,0.005449,1522713600,1.950857e+06,11036.00,2018-04-02 20:00:00
7,0.004620,0.005150,0.004506,0.005150,1522800000,1.947114e+06,8995.67,2018-04-03 20:00:00
8,0.004920,0.004920,0.004424,0.004653,1522886400,2.439877e+06,12004.19,2018-04-04 20:00:00
9,0.004370,0.004778,0.004111,0.004778,1522972800,2.988759e+06,13060.87,2018-04-05 20:00:00


In [37]:
acat.describe()

,close,high,low,open,time,volumefrom,volumeto
count,197.000000,197.000000,197.000000,197.000000,1.970000e+02,1.970000e+02,1.970000e+02
mean,0.004284,0.004654,0.003978,0.004322,1.530662e+09,6.892996e+07,2.275241e+05
std,0.003162,0.003390,0.002916,0.003184,4.925937e+06,1.548132e+08,4.846015e+05
min,0.000621,0.000679,0.000592,0.000622,1.522195e+09,1.894161e+05,1.345200e+02
25%,0.001080,0.001332,0.000946,0.001068,1.526429e+09,2.943611e+06,7.027390e+03
50%,0.003543,0.003895,0.003431,0.003609,1.530662e+09,8.090060e+06,3.419684e+04
75%,0.006895,0.007239,0.006344,0.006665,1.534896e+09,5.193754e+07,2.648939e+05
max,0.013500,0.013930,0.011460,0.012800,1.539130e+09,8.228835e+08,4.147192e+06


In [39]:
acath= acat['high'] - acat['low']


In [40]:
acat.insert(8, 'range', acath, allow_duplicates=False)


In [41]:
acat.to_csv('acat.csv')

In [42]:
btcd= daily_price_historical('BTC', 'USD')
print('Max length = %s' % len(btcd))
print('Max time = %s' % (btcd.timestamp.max() - btcd.timestamp.min()))
btcd

Max length = 3008
Max time = 3007 days 00:00:00


,close,high,low,open,time,volumefrom,volumeto,timestamp
0,0.04951,0.04951,0.04951,0.04951,1279324800,20.00,9.902000e-01,2010-07-16 20:00:00
1,0.08584,0.08585,0.05941,0.04951,1279411200,75.01,5.090000e+00,2010-07-17 20:00:00
2,0.08080,0.09307,0.07723,0.08584,1279497600,574.00,4.966000e+01,2010-07-18 20:00:00
3,0.07474,0.08181,0.07426,0.08080,1279584000,262.00,2.059000e+01,2010-07-19 20:00:00
4,0.07921,0.07921,0.06634,0.07474,1279670400,575.00,4.226000e+01,2010-07-20 20:00:00
5,0.05050,0.08181,0.05050,0.07921,1279756800,2160.00,1.297800e+02,2010-07-21 20:00:00
6,0.06262,0.06767,0.05050,0.05050,1279843200,2402.50,1.410700e+02,2010-07-22 20:00:00
7,0.05454,0.06161,0.05049,0.06262,1279929600,496.32,2.673000e+01,2010-07-23 20:00:00
8,0.05050,0.05941,0.05050,0.05454,1280016000,1551.48,8.506000e+01,2010-07-24 20:00:00
9,0.05600,0.05600,0.05000,0.05050,1280102400,877.00,4.691000e+01,2010-07-25 20:00:00


In [43]:
btcd.to_csv('btcd.csv')

In [44]:
ast = daily_price_historical('AST', 'USD')
print('Max length = %s' % len(ast))
print('Max time = %s' % (ast.timestamp.max() - ast.timestamp.min()))
ast.tail()
astt=ast.tail()

Max length = 414
Max time = 413 days 00:00:00


In [45]:
trace0 = go.Scatter(
    x = neo['timestamp'],
    y = neo['high'],
    name = 'NEO',
    line = dict(
        color = ('rgb(205, 12, 24)'),
        width = 4,
        dash = 'dash') # dash options include 'dash', 'dot', and 'dashdot'
)

trace1 = go.Scatter(
    x = acat['timestamp'],
    y = acat['high'],
    name = 'ACAT',
    line = dict(
        color = ('rgb(205, 12, 24)'),
        width = 4,
        dash = 'dash') # dash options include 'dash', 'dot', and 'dashdot'
)



trace2 = go.Scatter(
    x = dbc['timestamp'],
    y = dbc['high'],
    name = 'DBC',
    line = dict(
        color = ('rgb(205, 12, 24)'),
        width = 4,
        dash = 'dash') # dash options include 'dash', 'dot', and 'dashdot'
)

data = [trace0, trace1, trace2]

# Edit the layout
layout = dict(title = 'NEP-5 Historical Daily Prices',
              xaxis = dict(title = 'Date'),
              yaxis = dict(title = '$ USD'),
              )

fig2 = dict(data=data, layout=layout)
py.plot(fig2, filename='styled-line')

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/plotly/offline/offline.py:466: UserWarning:

Your filename `styled-line` didn't end with .html. Adding .html to the end of your file.



'file:///Users/lindsay_richman/styled-line.html'

## Historical Hourly Price

In [46]:
def hourly_price_historical(symbol, comparison_symbol, limit, aggregate, exchange=''):
    url = 'https://min-api.cryptocompare.com/data/histohour?fsym={}&tsym={}&limit={}&aggregate={}'\
            .format(symbol.upper(), comparison_symbol.upper(), limit, aggregate)
    if exchange:
        url += '&e={}'.format(exchange)
    page = requests.get(url)
    data = page.json()['Data']
    df = pd.DataFrame(data)
    df['timestamp'] = [datetime.datetime.fromtimestamp(d) for d in df.time]
    return df


In [47]:
time_delta = 1 # Bar width in hours
btc_h = hourly_price_historical('BTC', 'USD', 9999, time_delta)
print('Max length = %s' % len(btc_h))
print('Max time = %s' % (btc_h.timestamp.max() - btc_h.timestamp.min()))
btc_h

Max length = 2001
Max time = 83 days 08:00:00


,close,high,low,open,time,volumefrom,volumeto,timestamp
0,7425.90,7449.13,7416.41,7421.06,1532001600,3127.67,23374166.45,2018-07-19 08:00:00
1,7485.19,7515.68,7421.15,7425.90,1532005200,5071.46,37967728.26,2018-07-19 09:00:00
2,7460.13,7487.71,7444.25,7485.19,1532008800,4676.91,34999723.53,2018-07-19 10:00:00
3,7397.11,7482.35,7377.30,7461.12,1532012400,5242.04,39035941.05,2018-07-19 11:00:00
4,7374.74,7421.19,7363.45,7397.05,1532016000,3787.26,28098557.48,2018-07-19 12:00:00
5,7429.51,7435.95,7373.80,7374.70,1532019600,4868.04,36122659.52,2018-07-19 13:00:00
6,7424.44,7454.72,7414.76,7429.92,1532023200,2795.02,20822548.31,2018-07-19 14:00:00
7,7457.54,7493.08,7422.98,7424.20,1532026800,3634.37,27191948.36,2018-07-19 15:00:00
8,7426.90,7487.88,7426.90,7457.54,1532030400,2546.44,19105837.54,2018-07-19 16:00:00
9,7441.18,7467.21,7427.11,7428.18,1532034000,1449.19,10874873.71,2018-07-19 17:00:00


In [48]:
time_delta = 1 # Bar width in hours
ast_h = hourly_price_historical('AST', 'USD', 9999, time_delta)
print('Max length = %s' % len(ast_h))
print('Max time = %s' % (ast_h.timestamp.max() - ast_h.timestamp.min()))


Max length = 2001
Max time = 83 days 08:00:00


In [49]:
acat_h = hourly_price_historical('ACAT', 'USD', 9999, time_delta)
print('Max length = %s' % len(acat_h))
print('Max time = %s' % (acat_h.timestamp.max() - acat_h.timestamp.min()))

Max length = 2001
Max time = 83 days 08:00:00


In [50]:
acat_h.to_csv('acat_h.csv')

In [51]:
hpb_h = hourly_price_historical('HPB', 'USD', 9999, time_delta)
print('Max length = %s' % len(hpb_h))
print('Max time = %s' % (hpb_h.timestamp.max() - hpb_h.timestamp.min()))

Max length = 2001
Max time = 83 days 08:00:00


In [52]:
gnt_h = hourly_price_historical('GNT', 'USD', 9999, time_delta)
print('Max length = %s' % len(gnt_h))
print('Max time = %s' % (gnt_h.timestamp.max() - gnt_h.timestamp.min()))

Max length = 2001
Max time = 83 days 08:00:00


In [53]:
bat_h = hourly_price_historical('BAT', 'USD', 9999, time_delta)
print('Max length = %s' % len(bat_h))
print('Max time = %s' % (bat_h.timestamp.max() - bat_h.timestamp.min()))

Max length = 2001
Max time = 83 days 08:00:00


In [54]:
eos_h = hourly_price_historical('EOS', 'USD', 9999, time_delta)
print('Max length = %s' % len(eos_h))
print('Max time = %s' % (eos_h.timestamp.max() - eos_h.timestamp.min()))

Max length = 2001
Max time = 83 days 08:00:00


In [26]:
kin_h = hourly_price_historical('KIN', 'USD', 9999, time_delta)
print('Max length = %s' % len(kin_h))
print('Max time = %s' % (kin_h.timestamp.max() - kin_h.timestamp.min()))

Max length = 2001
Max time = 83 days 08:00:00


In [27]:
import plotly.plotly as py
from plotly.graph_objs import *

trace0 = go.Scatter(
    x = btc_h['timestamp'],
    y = btc_h['high'],
    name = 'BTC High',
    line = dict(
        color = ('rgb(0,110,197)'),
        width = 4,
        )
)


trace1 = go.Scatter(
    x = ast_h['timestamp'],
    y = ast_h['high'],
    name = ' AST High',
    line = dict(
        color = ('rgb(120,194,195)'),
        width = 4)
)

trace2 = go.Scatter(
    x = hpb_h['timestamp'],
    y = hpb_h['high'],
    name = 'HPB High',
    line = dict(
        color = ('rgb(205, 12, 24)'),
        width = 4) 
)

trace4 = go.Scatter(
    x = kin_h['timestamp'],
    y = kin_h['high'],
    name = 'KIN High',
    line = dict(
        color = ('rgb(226,206,65)'),
        width = 4,
        ) # dash options include 'dash', 'dot', and 'dashdot'
)


trace6 = go.Scatter(
    x = bat_h['timestamp'],
    y = bat_h['high'],
    name = 'BAT High',
    line = dict(
        color = ('rgb(105,50,92)'),
        width = 4,
        ) # dash options include 'dash', 'dot', and 'dashdot'
)



data = [trace0, trace1, trace2, trace4, trace6]

layout = dict(
    title='<b>Alt Coin Prices',
    updatemenus=list([
        dict(
            active=0,
            yanchor='top',
            buttons=list([
                dict(
                    args=['visible', [True, True, True, True, True]],
                    label='All',
                    method='restyle',
                    
                ),
                dict(
                    args=['visible', [True, False, False, False, False]],
                    label='BTC',
                    method='restyle'
                ),
                dict(
                    args=['visible', [False, True, False, False, False]],
                    label='AST',
                    method='restyle'
                ),
                dict(
                    args=['visible', [False, False, True, False, False]],
                    label='HPB',
                    method='restyle'
                ),
                dict(
                    args=['visible', [False, False, False, True, False]],
                    label='KIN',
                    method='restyle'
                ),
                dict(
                    args=['visible', [False, False, False, False, True]],
                    label='BAT',
                    method='restyle'
                )
            ]),
        )
    ]),
    xaxis=dict(
        title="Date",
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label='1m',
                     step='month',
                     stepmode='backward'),
                dict(count=3,
                     label='3m',
                     step='month',
                     stepmode='backward'),
                dict(count=1,
                    label='1y',
                    step='year',
                    stepmode='backward'),
            ])
        ),
        rangeslider=dict(),
        type='date'
    ),
    yaxis=dict(title="$USD"),
)



fig = dict(data=data, layout=layout)
py.iplot(fig, filename='newnew')

## Historical Price by Minute

In [28]:
time_delta = 24 # Bar width in hours
df = hourly_price_historical('ACAT', 'USD', 9999, time_delta)
print('Max length = %s' % len(df))
print('Max time = %s' % (df.timestamp.max() - df.timestamp.min()))


Max length = 84
Max time = 83 days 00:00:00


In [29]:
hminute = go.Scatter(x=df['timestamp'], y=df['close'])
py.iplot([hminute])

In [30]:
def minute_price_historical(symbol, comparison_symbol, limit, aggregate, exchange=''):
    url = 'https://min-api.cryptocompare.com/data/histominute?fsym={}&tsym={}&limit={}&aggregate={}'\
            .format(symbol.upper(), comparison_symbol.upper(), limit, aggregate)
    if exchange:
        url += '&e={}'.format(exchange)
    page = requests.get(url)
    data = page.json()['Data']
    df = pd.DataFrame(data)
    df['timestamp'] = [datetime.datetime.fromtimestamp(d) for d in df.time]
    return df

In [31]:
time_delta = 1 # Bar width in minutes
df = minute_price_historical('BTC', 'USD', 9999, time_delta)
print('Max length = %s' % len(df))
print('Max time = %s' % (df.timestamp.max() - df.timestamp.min()))


Max length = 2001
Max time = 1 days 09:20:00


In [32]:
class Hashabledict(dict):
    def __hash__(self):
        return hash(frozenset(self))

## Coin List

In [5]:
def coin_list():
    url = 'https://www.cryptocompare.com/api/data/coinlist/'
    page = requests.get(url)
    data = page.json()['Data']
    return data

In [6]:
data = coin_list()
RenderJSON(data)


In [7]:
cointable=pd.DataFrame(data)
cointable=cointable.T
cointable

,Algorithm,BuiltOn,CoinName,FullName,FullyPremined,Id,ImageUrl,Name,PreMinedValue,ProofType,SmartContractAddress,SortOrder,Sponsored,Symbol,TotalCoinSupply,TotalCoinsFreeFloat,Url
ETH,Ethash,N/A,Ethereum,Ethereum (ETH),0,7605,/media/20646/eth_logo.png,ETH,N/A,PoW,N/A,2,False,ETH,0,N/A,/coins/eth/overview
LTC,Scrypt,N/A,Litecoin,Litecoin (LTC),0,3808,/media/19782/litecoin-logo.png,LTC,N/A,PoW,N/A,3,False,LTC,84000000,N/A,/coins/ltc/overview
DASH,X11,N/A,Dash,Dash (DASH),0,3807,/media/33842920/dash.png,DASH,N/A,PoW/PoS,N/A,4,False,DASH,22000000,N/A,/coins/dash/overview
XMR,CryptoNight-V7,N/A,Monero,Monero (XMR),0,5038,/media/19969/xmr.png,XMR,N/A,PoW,N/A,5,False,XMR,0,N/A,/coins/xmr/overview
NXT,PoS,N/A,Nxt,Nxt (NXT),1,1183,/media/20627/nxt.png,NXT,N/A,PoS/LPoS,N/A,6,False,NXT,1000000000,N/A,/coins/nxt/overview
DOGE,Scrypt,N/A,Dogecoin,Dogecoin (DOGE),0,4432,/media/19684/doge.png,DOGE,N/A,PoW,N/A,8,False,DOGE,N/A,N/A,/coins/doge/overview
ZEC,Equihash,N/A,ZCash,ZCash (ZEC),0,24854,/media/351360/zec.png,ZEC,N/A,PoW,N/A,9,False,ZEC,21000000,N/A,/coins/zec/overview
BTS,SHA-512,N/A,Bitshares,Bitshares (BTS),1,5039,/media/20705/bts.png,BTS,N/A,PoS,N/A,10,False,BTS,2511953117,N/A,/coins/bts/overview
DGB,Multiple,N/A,DigiByte,DigiByte (DGB),0,4430,/media/12318264/7638-nty_400x400.jpg,DGB,N/A,PoW,N/A,11,False,DGB,21000000000,N/A,/coins/dgb/overview
XRP,N/A,N/A,XRP,XRP (XRP),0,5031,/media/34477776/xrp.png,XRP,N/A,N/A,N/A,12,False,XRP,100000000000,N/A,/coins/xrp/overview


In [8]:
cointable['Algorithm'].value_counts()

N/A                     1647
Scrypt                   505
X11                      239
SHA256                   155
X13                       75
PoS                       72
CryptoNight               39
Quark                     29
SHA256D                   29
DPoS                      28
NeoScrypt                 25
Ethash                    24
Multiple                  22
Equihash                  21
X15                       19
NIST5                     19
Lyra2REv2                  8
QuBit                      8
POS 3.0                    8
XEVAN                      8
Lyra2RE                    7
Scrypt-n                   6
Dagger                     6
Dagger-Hashimoto           6
CryptoNight Heavy          5
Groestl                    5
Blake                      5
Lyra2Z                     5
Skein                      5
SHA-512                    5
                        ... 
Shabal256                  1
Whirlpool                  1
Slatechain                 1
Progressive-n 

In [9]:
list = [ [min, max] for min, max in zip(cointable.min(), cointable.max())]
list

[['1GB AES Pattern Search', 'vDPOS'],
 ['112392', 'N/A'],
 [' Arcade City', 'ÕpetFoundation'],
 [' Arcade City (ARC*)', 'ÕpetFoundation (OPET)'],
 ['0', '1'],
 ['100944', '925939'],
 ['$NOBS', 'xGOx'],
 ['N/A', 'N/A'],
 [' PoW/PoS', 'mFBA'],
 ['0x001f0aa5da15585e5b2305dbab2bac425ea71007',
  'QUb39Kx2qoMRnVsdwFjo31jZhQjP1bVH3P'],
 ['1', '999'],
 [False, False],
 ['007', 'redBUX'],
 [' 1000000000000000000', '\u200b1112000'],
 ['N/A', 'N/A'],
 ['/coins/007/overview', '/coins/zyd/overview']]

In [10]:
print(data.keys())

dict_keys(['ETH', 'LTC', 'DASH', 'XMR', 'NXT', 'DOGE', 'ZEC', 'BTS', 'DGB', 'XRP', 'BTCD', 'PPC', 'CRAIG', 'XBS', 'XPY', 'PRC', 'YBC', 'DANK', 'GIVE', 'KOBO', 'DT', 'CETI', 'SUP', 'XPD', 'GEO', 'CHASH', 'SPR', 'NXTI', 'WOLF', 'XDP', '2015', '42', 'AC', 'ACOIN', 'AERO', 'ALF', 'AGS', 'AMC', 'ALN', 'APEX', 'ARCH', 'ARG', 'ARI', 'AXR', 'BCX', 'BEN*', 'BET', 'BITB', 'BLU', 'BLK', 'BOST', 'BQC', 'MOON', 'ZET', 'SXC', 'QTL', 'ENRG', 'QRK', 'RIC', 'DGC', 'LIMX', 'BTB', 'CAIX', 'BTE', 'BTG*', 'BTM', 'BUK', 'CACH', 'CANN', 'CAP', 'CASH', 'CAT1', 'CBX', 'CCN', 'CIN', 'CINNI', 'CXC', 'CLAM', 'CLOAK', 'CLR', 'CMC', 'CNC', 'CNL', 'COMM', 'COOL', 'CRACK', 'CRC*', 'CRYPT', 'CSC', 'DEM', 'DMD', 'DRKC', 'DSB', 'DVC', 'EAC', 'EFL', 'ELC', 'EMC2', 'EMD', 'EXCL', 'EXE', 'EZC', 'FLAP', 'FC2', 'FFC', 'FIBRE', 'FRC', 'FLT', 'FRK', 'FRAC', 'FST', 'FTC', 'GDC', 'GLC', 'GLD', 'GLX', 'GLYPH', 'GML', 'GUE', 'HAL', 'HBN', 'HUC', 'HVC', 'HYP', 'ICB', 'IFC', 'IOC', 'IXC', 'JBS', 'JKC', 'JUDGE', 'KDC', 'KEY*', 'KGC',

In [15]:
for key, value in data.items():
    if value: 
        print(key, 'is the key for the value', value)

ETH is the key for the value {'Id': '7605', 'Url': '/coins/eth/overview', 'ImageUrl': '/media/20646/eth_logo.png', 'Name': 'ETH', 'Symbol': 'ETH', 'CoinName': 'Ethereum', 'FullName': 'Ethereum (ETH)', 'Algorithm': 'Ethash', 'ProofType': 'PoW', 'FullyPremined': '0', 'TotalCoinSupply': '0', 'BuiltOn': 'N/A', 'SmartContractAddress': 'N/A', 'PreMinedValue': 'N/A', 'TotalCoinsFreeFloat': 'N/A', 'SortOrder': '2', 'Sponsored': False}
LTC is the key for the value {'Id': '3808', 'Url': '/coins/ltc/overview', 'ImageUrl': '/media/19782/litecoin-logo.png', 'Name': 'LTC', 'Symbol': 'LTC', 'CoinName': 'Litecoin', 'FullName': 'Litecoin (LTC)', 'Algorithm': 'Scrypt', 'ProofType': 'PoW', 'FullyPremined': '0', 'TotalCoinSupply': '84000000', 'BuiltOn': 'N/A', 'SmartContractAddress': 'N/A', 'PreMinedValue': 'N/A', 'TotalCoinsFreeFloat': 'N/A', 'SortOrder': '3', 'Sponsored': False}
DASH is the key for the value {'Id': '3807', 'Url': '/coins/dash/overview', 'ImageUrl': '/media/33842920/dash.png', 'Name': 'D

In [12]:
data['DOGE']['Algorithm']

'Scrypt'

In [13]:
for key in list:
    if key in dict and value in data.itervalues():
        print (key)

TypeError: argument of type 'type' is not iterable

In [14]:
symbol_id_dict = {symb: int(d['Id']) for symb, d in data.items()}

In [ ]:
print(json.dumps(symbol_id_dict, indent=2))

## Live Coin Info

In [ ]:
def coin_snapshot_full_by_id(symbol, symbol_id_dict={}):
    if not symbol_id_dict:
        symbol_id_dict = {
            'BTC': 1182,
            'ETH': 7605,
            'LTC': 3808
        }
    symbol_id = symbol_id_dict[symbol.upper()]
    url = 'https://www.cryptocompare.com/api/data/coinsnapshotfullbyid/?id={}'\
            .format(symbol_id)
    page = requests.get(url)
    sdata = page.json()['Data']
    return sdata

In [ ]:
sdata

In [ ]:
socialdata = coin_snapshot_full_by_id('ETH', symbol_id_dict)
RenderJSON(data)

In [ ]:
data = live_social_status('NEO', symbol_id_dict)
RenderJSON(data)

In [ ]:
def live_social_status(symbol, symbol_id_dict={}):
    if not symbol_id_dict:
        symbol_id_dict = {
            'BTC': 1182,
            'ETH': 7605,
            'LTC': 3808
        }
    symbol_id = symbol_id_dict[symbol.upper()]
    url = 'https://www.cryptocompare.com/api/data/socialstats/?id={}'\
            .format(symbol_id)
    page = requests.get(url)
    socialdata = page.json()['Data']
    return socialdata

In [ ]:
socialdata = live_social_status('HPB',symbol_id_dict)
RenderJSON(data)
socialdata

In [ ]:
social=pd.DataFrame(socialdata)
social=social.T
social

In [ ]:
data2 = pd.DataFrame.from_dict(data, orient='columns')
data2['Twitter']['followers']

In [ ]:
import requests
url = 'https://min-api.cryptocompare.com/data/histominute' +\
        '?fsym=HPB' +\
        '&tsym=USD' +\
        '&limit=2000' +\
        '&aggregate=1'
response = requests.get(url)
data = response.json()['Data']

import pandas as pd
df2 = pd.DataFrame(data)
print(df2)

In [ ]:
### import requests
url = 'https://min-api.cryptocompare.com/data/histominute' +\
        '?fsym=AST' +\
        '&tsym=USD' +\
        '&limit=2000' +\
        '&aggregate=1'
response = requests.get(url)
data2 = response.json()['Data']

import pandas as pd
df3 = pd.DataFrame(data2)
print(df3)

In [ ]:
trace0 = go.Scatter(
    x = df2['time'],
    y = df2['high'],
    name = 'HPB High',
    line = dict(
        color = ('rgb(205, 12, 24)'),
        width = 4)
)
trace1 = go.Scatter(
    x = df3['time'],
    y = df3['high'],
    name = 'AST High',
    line = dict(
        color = ('rgb(120,194,195)'),
        width = 4,)
)

data = [trace0, trace1]

# Edit the layout
layout = dict(title = 'HPB and AST Prices',
              xaxis = dict(title = 'Time'),
              yaxis = dict(title = 'USD'),
              )

fig = dict(data=data, layout=layout)
py.iplot(fig, filename='styled-line-hpb-ast')

In [ ]:
token = ['AST', 'HBP']
score = [1,6,2,8,2,9,4,5,1,5,2,8]

aggs = ["count","sum","avg","median","mode","rms","stddev","min","max","first","last"]

agg = []
agg_func = []
for i in range(0, len(aggs)):
    agg = dict(
        args=['transforms[0].aggregations[0].func', aggs[i]],
        label=aggs[i],
        method='restyle'
    )
    agg_func.append(agg)


data = [dict(
  type = 'scatter',
  x = subject,
  y = score,
  mode = 'markers',
  transforms = [dict(
    type = 'aggregate',
    groups = subject,
    aggregations = [dict(
        target = 'y', func = 'sum', enabled = True)
    ]
  )]
)]

layout = dict(
  title = '<b>Plotly Aggregations</b><br>use dropdown to change aggregation',
  xaxis = dict(title = 'Subject'),
  yaxis = dict(title = 'Score', range = [0,22]),
  updatemenus = [dict(
        x = 0.85,
        y = 1.15,
        xref = 'paper',
        yref = 'paper',
        yanchor = 'top',
        active = 1,
        showactive = False,
        buttons = agg_func
  )]
)

iplot({'data': data,'layout': layout}, validate=False)


In [ ]:
def social_list():
    url = 'https://www.cryptocompare.com/api/data/coinlist/'
    page = requests.get(url)
    data = page.json()['Data']
    return data

In [ ]:
from plotly.graph_objs import FigureWidget

# core
import io
import base64 
import time

# pandas
import pandas as pd

# numpy
import numpy as np

# scikit learn
from sklearn import datasets

# datashader
import datashader as ds
import datashader.transfer_functions as tf
from datashader.colors import inferno

In [ ]:
num_copies = 7000 # 1,050,000 rows

iris_data = datasets.load_iris()
feature_names = [name.replace(' (cm)', '').replace(' ', '_') for name in iris_data.feature_names]
iris_df_orig = pd.DataFrame(iris_data.data, columns=feature_names)
target_orig = iris_data.target + 1

# frame of features
iris_df = pd.concat(
    np.random.normal(scale=0.2, size=iris_df_orig.shape) + iris_df_orig for i in range(num_copies)
).reset_index(drop=True)

# array of targets
target = [t for i in range(num_copies) for t in target_orig]

# dataframe that includes target as categorical
iris_target_df = pd.concat([iris_df, pd.Series(target, name='target', dtype='category')], axis=1)

iris_df.describe()

In [ ]:
def gen_ds_image(x_range, y_range, plot_width, plot_height):
    if x_range is None or y_range is None or plot_width is None or plot_height is None:
        return None
    
    cvs = ds.Canvas(x_range=x_range, y_range=y_range, plot_height=plot_height, plot_width=plot_width)
    agg_scatter = cvs.points(iris_target_df, 
                         'sepal_length', 'sepal_width', 
                          ds.count_cat('target'))
    img = tf.shade(agg_scatter)
    img = tf.dynspread(img, threshold=0.95, max_px=5, shape='circle')
    
    return img.to_pil()

In [ ]:
x_range=[3, 10]
y_range=[0, 6]
plot_height=500
plot_width=700

In [ ]:
initial_img = gen_ds_image(x_range, y_range, plot_width, plot_height)
initial_img

In [ ]:
f.layout.images = [dict(
    source = initial_img,  # plotly now performs auto conversion of PIL image to png data URI
    xref = "x",
    yref = "y",
    x = x_range[0],
    y = y_range[1],
    sizex = x_range[1] - x_range[0],
    sizey = y_range[1] - y_range[0],
    sizing = "stretch",
    layer = "below")]

In [ ]:
def update_ds_image(layout, x_range, y_range, plot_width, plot_height):
    img = f.layout.images[0]
    
    # Update with batch_update so all updates happen simultaneously
    with f.batch_update():
        img.x = x_range[0]
        img.y = y_range[1]
        img.sizex = x_range[1] - x_range[0]
        img.sizey = y_range[1] - y_range[0]
        img.source = gen_ds_image(x_range, y_range, plot_width, plot_height)

# Install callback to run exactly once if one or more of the following properties changes
#  - xaxis range
#  - yaxis range
#  - figure width
#  - figure height
f.layout.on_change(update_ds_image, 'xaxis.range', 'yaxis.range', 'width', 'height')

In [ ]:
f.layout.dragmode = 'zoom'
f

In [ ]:
import plotly.dashboard_objs as dashboard

import IPython.display
from IPython.display import Image

plotly.tools.set_credentials_file(username='lindsayrichman', api_key='ildSRBa101xXGmiL63uy')
import plotly.plotly as py
plotly.tools.set_config_file(world_readable=False,
                             sharing='private')

my_dboard = dashboard.Dashboard()
my_dboard.get_preview()

In [ ]:
my_dboard.get_preview()

In [ ]:
import plotly.graph_objs as go
import plotly.plotly as py

import numpy as np

colorscale = [[0, '#FAEE1C'], [0.33, '#F3558E'], [0.66, '#9C1DE7'], [1, '#581B98']]
trace1 = go.Scatter(
    y = np.random.randn(500),
    mode='markers',
    marker=dict(
        size=16,
        color = np.random.randn(500),
        colorscale=colorscale,
        showscale=True
    )
)
data = [trace1]
url_1 = py.plot(data, filename='scatter-for-dashboard', auto_open=False)
py.iplot(data, filename='scatter-for-dashboard')

In [ ]:
import plotly.plotly as py
import plotly.graph_objs as go

import numpy as np

x0 = np.random.randn(50)
x1 = np.random.randn(50) + 2
x2 = np.random.randn(50) + 4
x3 = np.random.randn(50) + 6

colors = ['#FAEE1C', '#F3558E', '#9C1DE7', '#581B98']

trace0 = go.Box(x=x0, marker={'color': colors[0]})
trace1 = go.Box(x=x1, marker={'color': colors[1]})
trace2 = go.Box(x=x2, marker={'color': colors[2]})
trace3 = go.Box(x=x3, marker={'color': colors[3]})
data = [trace0, trace1, trace2, trace3]

url_2 = py.plot(data, filename='box-plots-for-dashboard', sharing='secret', auto_open=False)
py.iplot(data, filename='box-plots-for-dashboard')

In [ ]:
def fileId_from_url(url):
    """Return fileId from a url."""
    index = url.find('~')
    fileId = url[index + 1:]
    local_id_index = fileId.find('/')

    share_key_index = fileId.find('?share_key')
    if share_key_index == -1:
        return fileId.replace('/', ':')
    else:
        return fileId[:share_key_index].replace('/', ':')

def sharekey_from_url(url):
    """Return the sharekey from a url."""
    index = url.find('share_key=')
    return url[index + len('share_key='):]

fileId_1 = fileId_from_url(url_1)
fileId_2 = fileId_from_url(url_2)

box_a = {
    'type': 'box',
    'boxType': 'plot',
    'fileId': fileId_1,
    'title': 'scatter-for-dashboard',
}

text_for_box = """ 
## Distributions: 


#### Scatter Plot
1. Ranging 0 - 500
2. Even distribution

#### Box Plot
1. Similar Range
2. Outliers present in trace 1 and trace 3

You can view more markdown tips [here](https://daringfireball.net/projects/markdown/syntax).
"""

box_b = {
    'type': 'box',
    'boxType': 'text',
    'text': text_for_box,
    'title': 'Markdown Options for Text Box'
}

box_c = {
    'type': 'box',
    'boxType': 'plot',
    'fileId': fileId_2,
    'title': 'box-for-dashboard',
    'shareKey': sharekey_from_url(url_2)
}

print(my_dboard.insert(box_a))

In [ ]:
my_dboard.insert(box_b, 'above', 1)

In [ ]:
my_dboard.insert(box_c, 'left', 2)

In [ ]:
my_dboard.get_box(1)

In [ ]:
my_dboard.get_box(1)['title'] = 'a new title'
my_dboard.get_box(1)

In [ ]:
my_dboard['settings']['title'] = 'My First Dashboard with Python'

In [ ]:
print(my_dboard)

In [99]:
pred=pd.read_csv("acat_prediction_low.csv")
pred

,inst#,actual,predicted,error,timestamp
0,1,0.009,0.009,0.000,3/27/18
1,2,0.008,0.008,0.000,3/28/18
2,3,0.003,0.004,0.000,3/29/18
3,4,0.005,0.005,0.001,3/30/18
4,5,0.005,0.006,0.001,3/31/18
5,6,0.001,0.001,0.000,4/1/18
6,7,0.003,0.003,0.000,4/2/18
7,8,0.003,0.003,0.000,4/3/18
8,9,0.003,0.003,0.000,4/4/18
9,10,0.003,0.003,0.000,4/5/18


In [96]:
predly = go.Scatter(x=pred['inst#'], y=pred['actual'])
py.plot([predly])

'https://plot.ly/~lindsayrichman/643'

In [102]:
trace0 = go.Scatter(
    x = pred['timestamp'],
    y = pred['actual'],
    mode = 'markers+lines',
    name = 'markers'
)


data = [trace0]
py.iplot(data, filename='scatter-mode')



In [140]:
import plotly.plotly as py
import plotly.graph_objs as go


trace_actual_low = go.Scatter(
                mode='markers',
                x=pred['timestamp'],
                y=pred['actual'],
                name = "Actual Low Price",
                marker = dict(
                    color = 'rgb(17, 157, 255)',
                    size = 5,
                    line = dict(color = 'rgb(231, 99, 250)',
                                width = 2
                               )
                )
)
                

trace_predicted_low = go.Scatter(
                mode='markers',
                x=pred_high['timestamp'],
                y=pred_high['predicted'],
                name = "Predicted Low Price",
                marker = dict(color = '#7F7F7F'),
                opacity = 0.8)

trace_actual_high = go.Scatter(
                mode='markers',
                x=pred_high['timestamp'],
                y=pred_high['actual'],
                name = "Actual High Price",
                marker = dict(color = '#78c2c3'),
                opacity = 0.8)

trace_predicted_high = go.Scatter(
                mode='markers',
                x=pred_high['timestamp'],
                y=pred_high['actual'],
                name = "Actual High Price",
                marker = dict(color = '#c41230'),
                opacity = 0.8)



data = [trace_actual_low, trace_predicted_low, trace_actual_high, trace_predicted_high]

layout = dict(
    title = "ACAT Prices: Actual and Predicted",
)

fig = dict(data=data, layout=layout)
py.iplot(fig, filename = "ACAT")

In [117]:
pred_high=pd.read_csv("acat_high_pred.csv")
pred_high

,inst#,actual,predicted,error,timestamp
0,1,0.011,0.010,0.000,3/27/18
1,2,0.009,0.009,0.000,3/28/18
2,3,0.004,0.004,0.000,3/29/18
3,4,0.005,0.006,0.001,3/30/18
4,5,0.006,0.006,0.001,3/31/18
5,6,0.001,0.001,0.000,4/1/18
6,7,0.003,0.003,0.000,4/2/18
7,8,0.003,0.003,0.000,4/3/18
8,9,0.004,0.004,0.000,4/4/18
9,10,0.004,0.004,0.000,4/5/18
